# Batch Result Download
Download a batch of Conveyal regional analysis results

Setup requires:
 - Completed regional analysis results, viewable in the Conveyal user interface
 - JSON with array values for the keys `analysisIds`, `cutoffs`, `pointSetId`, and `percentile`, plus the `regionId` saved to `config/regionalAnalysisParameters.json`. The values in the arrays can be derived from the URL of a regional analysis results page in the Conveyal user interface: `https://analysis.conveyal.com/regions/[regionId]/regional/[analysisId]?cutoff=[cutoff]&pointSetId=[pointSetId]&percentile=[percentile]`
 - A current Conveyal token (e.g. 'bearer 1234abcd...') saved at `config/.auth`, based on the `idToken` shown at https://analysis.conveyal.com/session (for logged in users).
 - Optionally, a session cookie copied from browser DevTools saved at `config/.cookie`

After the setup cell in this notebook, there are cells to: 
 - Optionally, fetch the names of regional analyses and destination pointset layers
 - Download all combinations of analyses, cutoffs, destination pointsets, and percentiles

## Setup

In [ ]:
import requests
import json
import urllib
import os

config = json.load(open('config/regionalAnalysisParameters.json'))

# Authorization header copied from DevTools Network request or https://analysis.conveyal.com/session
token = open('config/.auth').readline().strip()
headers = {
    'Authorization': token
}
resultUrl = 'https://api.conveyal.com/api/regional/'

namesFetched = False;

# Cookie copied from browser Devtools Network request header
cookie = open('config/.cookie').readline().strip()
dbHeaders = {
    'Cookie': cookie
}
dbUrl = 'https://analysis.conveyal.com/api/db/'

## Fetch names
Optional, requires saved cookie

In [ ]:
analysesRequest = requests.get(dbUrl + 'regional-analyses?options=' + urllib.parse.urlencode({"projection":{"request.scenario.modifications":0}}), headers = dbHeaders)
analyses = analysesRequest.json()

opportunitiesRequest = requests.get(dbUrl + 'opportunityDatasets?', headers = dbHeaders)
opportunities = opportunitiesRequest.json()
destinations = {}

for destinationPointSetId in config['pointSetIds']:
    destinationPointSetName = list(filter(lambda x:x["_id"] == destinationPointSetId, opportunities))[0]['name']
    destinations[destinationPointSetId] = destinationPointSetName
    if not os.path.exists('results/To ' + destinationPointSetName):
        os.makedirs('results/To ' + destinationPointSetName)

namesFetched = True;

## Download regional analysis result geotiff files
For all analysis, cutoff, percentile, and destination combinations

In [ ]:
# Loop over analysis ids
for analysisId in config['analysisIds']:
    
    analysisName = analysisId
    if namesFetched:
        analysisProperties = list(filter(lambda x:x['_id']==analysisId, analyses))[0]
        with open('results/' + analysisId + '.json', 'w') as f:
            json.dump(analysisProperties, f)
        analysisName = analysisProperties['name']
    print('Processing ' + analysisName)
    
    # Loop over cutoffs, percentiles, and destination opportunity pointsets
    for cutoff in config['cutoffs']:
        for percentile in config['percentiles']:
            for destinationPointSetId in config['pointSetIds']:
                
                destinationPointSetName = destinationPointSetId
                if namesFetched:
                    destinationPointSetName = destinations[destinationPointSetId]
                
                gridQuery = urllib.parse.urlencode({'cutoff': cutoff, 'percentile': percentile, 'destinationPointSetId': destinationPointSetId})
                gridUrl = resultUrl + analysisId + '/grid/geotiff?' + gridQuery
                
                # Request a signed S3 url from the Conveyal API
                r = requests.get(gridUrl, headers = headers, verify = False)
    
                if r.status_code == 403:
                    print('Unauthorized access. Your authorization token may be invalid or expired.')
        
                elif r.status_code != 200:
                    print('Error: ' + r.text)
        
                else:
                    # From the signed S3 url, fetch the grid as a geotiff
                    grid = requests.get(r.json()['url'], verify = False)
                    # Save response from Conveyal Analysis to a local .geotiff file
                    with open('results/To ' + destinationPointSetName + '/' + ('-').join((analysisId, str(cutoff) + 'min', str(percentile) + 'pct')) + '.geotiff', 'wb') as f:
                        for chunk in grid.iter_content(chunk_size=128):
                            f.write(chunk)
                        f.close()